In [ ]:
!nvidia-smi

Sun May 23 04:16:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
'Colab Notebooks'   yolov3


#clone , configuaration and compile darknet

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15050, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 15050 (delta 16), reused 34 (delta 14), pack-reused 15002
Receiving objects: 100% (15050/15050), 13.48 MiB | 16.59 MiB/s, done.
Resolving deltas: 100% (10215/10215), done.


In [ ]:
#configuration
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile


/content/waste/darknet


In [ ]:
!make

# configuration Yolov3.cfg file

In [ ]:
#Make a copy of yolov3.cfg
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
#change lines in yolov3.cfg file
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches=500200/max_batches=32000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=16@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=16@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=16@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=63@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=63@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=63@' cfg/yolov3_training.cfg

# create .names and .data files

In [ ]:
!echo -e 'dog\nperson\ncat\ntv\ncar\nmeatballs\nmarinara sauce\ntomato soup\nchicken noodle soup\nfrench onion soup\nchicken breast\nribs\npulled pork\nhamburger\ncavity\nLicense Plate' >data/obj.names
!echo -e 'classes= 16\ntrain = data/train.txt\nvalid=data/test.txt\nnames=data/obj.names\nbackup=/mydrive/yolov3' >data/obj.data

# Save Yolov3_training.cfg and obj.names files in google drive

In [ ]:
!cp cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_testing.cfg
!cp data/obj.names /mydrive/yolov3/classes.txt

# Create a folder and unzip image dataset

In [ ]:
!mkdir data/obj
!unzip /mydrive/yolov3/images.zip -d data/obj

# create train.txt file

In [ ]:
import glob
images_list = glob.glob("data/obj/*.png")
with open("data/train.txt","w") as f:
  f.write("\n".join(images_list))

# Download pre-trainned weights for the convolution layers file

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-05-23 06:05:59--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74.4’

darknet53.conv.74.4 100%[===================>] 154.96M  63.2MB/s    in 2.5s    

2021-05-23 06:06:02 (63.2 MB/s) - ‘darknet53.conv.74.4’ saved [162482580/162482580]



# Start Training

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show
#incase your  trainings stopped then to restart from the last stage use below code
#!./darknet detector train data/obj.data cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_training_last.weights -dont_show

 CUDA-version: 11000 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
yolov3_training
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 1666, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 S